In [1]:
!pip install fastai

^C


  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached scipy-1.14.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
     ---------------------------------------- 0.0/149.4 kB ? eta -:--:--
     -------------------------------------- 149.4/149.4 kB 9.3 MB/s eta 0:00:00
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl.metadata (34 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
     ---------------------------------------- 0.0/167.0 kB ? eta -:--:--
     ---------------------------------------- 167.0/167.0 kB ? eta 0:00:00
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB ? eta 0:00:00
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import shutil
import random

def create_validation_set(train_dir, val_dir, split_ratio=0.2):
    # Create validation directory if it doesn't exist
    os.makedirs(val_dir, exist_ok=True)

    # Iterate over each label folder in the train directory
    for label in os.listdir(train_dir):
        label_path = os.path.join(train_dir, label)

        # Check if it is a directory (label folder)
        if os.path.isdir(label_path):
            images = os.listdir(label_path)
            # random.shuffle(images)  # Shuffle the images

            # Calculate the number of validation images (20% of total)
            val_size = int(len(images) * split_ratio)

            # Select the validation images
            val_images = images[:val_size]

            # Create label subdirectory in the validation directory
            os.makedirs(os.path.join(val_dir, label), exist_ok=True)

            # Move validation images to validation directory
            for image in val_images:
                shutil.move(os.path.join(label_path, image), os.path.join(val_dir, label, image))

if __name__ == "__main__":
    # Define the train directory and the validation directory
    train_directory = './screen_caps/train'  # Example: 'train'
    val_directory = './screen_caps/valid'      # Example: 'val'

    # Create the validation set
    create_validation_set(train_directory, val_directory, split_ratio=0.2)


pretrained model

In [2]:
from fastai.vision.all import *

path = Path("./screen_caps")

# Create a DataLoader for training
dls = ImageDataLoaders.from_folder(path, train='train', valid='valid', 
                                   item_tfms=Resize(224), batch_tfms=aug_transforms(), 
                                   bs=64)



# Define the model using FastAI's `cnn_learner` function
# ResNet is a commonly used architecture. You can replace resnet34 with any other architecture
learner = cnn_learner(dls, resnet34, metrics=accuracy)


# Train the model
learner.fine_tune(10)  # Train for 10 epochs

learner.save('model')

c:\Users\Jeffr\Documents\School\IT\Deep Learning\CVC_challenge\CVC_challenge_env\Lib\site-packages\fastai\vision\learner.py:303: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")


epoch,train_loss,valid_loss,accuracy,time
0,2.280041,1.816031,0.284483,01:18


epoch,train_loss,valid_loss,accuracy,time
0,2.052748,1.652797,0.336207,01:27
1,1.956285,1.552629,0.327586,01:25
2,1.881240,1.995276,0.252874,01:22
3,1.810206,1.515904,0.310345,01:22
4,1.707793,1.384849,0.405172,01:22
5,1.611469,1.392330,0.347701,01:22
6,1.545098,1.276369,0.387931,01:23
7,1.434225,1.266422,0.390805,01:22
8,1.371032,1.275477,0.399425,01:22
9,1.331322,1.273682,0.402299,01:22


Path('screen_caps/models/model.pth')

In [4]:
from fastai.vision.all import *
from mss import mss
import numpy as np
import cv2
import time
import pygetwindow as gw
from vgamepad import VDS4Gamepad

# Set up the screen capture and gamepad simulation
class ScreenCapture:
    def __init__(self, game_area=None):
        self.capture = mss()  # Correct mss initialization
        if game_area is None:
            game_area = self.get_game_window_coords()
            if game_area is None:
                raise ValueError("Game window not found.")
            self.game_area = {
                "left": game_area[0], 
                "top": game_area[1], 
                "width": game_area[2], 
                "height": game_area[3]
            }
        else:
            self.game_area = game_area

    def capture_frame(self):
        gamecap = np.array(self.capture.grab(self.game_area))
        return gamecap

    def get_game_window_coords(self, title="Forza Horizon 4"):
        game_window = gw.getWindowsWithTitle(title)
        if game_window:
            game_window = game_window[0]
            return game_window.left, game_window.top, game_window.width, game_window.height
        return None

# Set up the gamepad simulator
class GamepadSimulator:
    def __init__(self):
        self.gamepad = VDS4Gamepad()

    def set_input(self, lx, lt, rt):
        """Simulate controller inputs based on the model's prediction."""
        # Left joystick for steering (LX)
        self.gamepad.left_joystick(int((lx + 1) * 127), 128)
        
        # Left trigger (LT) for brake
        # self.gamepad.left_trigger(int(lt * 255))
        
        # Right trigger (RT) for gas
        self.gamepad.right_trigger(int(rt * 255))
        
        # Update the gamepad state
        self.gamepad.update()

# Load the trained model
def load_model():
    learner = cnn_learner(dls, resnet34, metrics=accuracy)
    learner.load('model')  # Load the saved model
    return learner

# Use the model to predict actions and simulate gamepad inputs
def drive_game(learner, screen_capture, gamepad_simulator, duration=60):
    start_time = time.time()
    
    while time.time() - start_time < duration:
        # Capture the game screen
        frame = screen_capture.capture_frame()
        
        # Convert the frame to a PIL image
        pil_img = PILImage.create(frame)
        if pil_img.mode == 'RGBA':
            pil_img = pil_img.convert('RGB')
        
        # Predict the next action using the trained model
        pred, _, _ = learner.predict(pil_img)
        
        # Extract the predicted action (you can modify this part to map to your controller actions)
        action = pred
        print(f"Predicted Action: {action}")
        
        # Simulate the controller inputs based on the prediction (implement your logic)
        if action == 'steer_left':
            gamepad_simulator.set_input(lx=-1, lt=0, rt=0)
        elif action == 'steer_right':
            gamepad_simulator.set_input(lx=1, lt=0, rt=0)
        # elif action == 'brake':
        #     gamepad_simulator.set_input(lx=0, lt=1, rt=0)
        elif action == 'gas':
            gamepad_simulator.set_input(lx=0, lt=0, rt=1)
        
        # Sleep for a short period to control the simulation speed
        time.sleep(0.1)

if __name__ == '__main__':
    # Initialize components
    screen_capture = ScreenCapture()
    gamepad_simulator = GamepadSimulator()
    learner = load_model()

    # Start driving the game for 60 seconds (you can adjust the duration)
    drive_game(learner, screen_capture, gamepad_simulator, duration=60)


Predicted Action: gas


Predicted Action: gas


Predicted Action: gas


Predicted Action: gas


Predicted Action: gas


...

Predicted Action: steer_left


Predicted Action: gas


Predicted Action: gas


Predicted Action: steer_left


Predicted Action: steer_left


custom model

In [ ]:
from fastai.vision.all import *
from fastbook import *
from fastai.vision.widgets import *
import torch.nn as nn
from PIL import Image


path = Path("path_to_your_dataset")

im = Image.open(path + 'cat/cat.1.jpg')
im.to_thumb(128,128)

filenames = get_image_files(path)
filenames

#virify images
failed = verify_images(filenames)
failed

splitter=RandomSplitter(valid_pct=0.2, seed=42)
item_tfms=Resize(128)

# Define a custom CNN
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, stride=3)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=3)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128*output_dim_x*output_dim_y, 5)  # Adjust output dim after flatten

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flatten(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

# Create data loaders
dls = ImageDataLoaders.from_folder(path, item_tfms=Resize(224), batch_tfms=aug_transforms(), bs=64)

# Wrap your model into a Learner
learner = Learner(dls, MyModel(), metrics=accuracy)

# Train the custom model
learner.fit_one_cycle(10)